In [ ]:
import os, shutil, gc

import hda
from webdav3.client import Client
from getpass import getpass

import xarray as xr
import numpy as np
import cf_xarray

from datetime import datetime

from cdo import *
cdo = Cdo()
cdo.cleanTempDir()

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm
import colormaps as cmo

from cartopy import crs as ccrs
from cartopy import feature as cf
from shapely import geometry
from shapely.geometry import shape

import zipfile

from rich.jupyter import print
from rich.markdown import Markdown

import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm


In [ ]:
download_dir = os.path.join(os.path.expanduser('~'),"edskywalker", "downloaded-data")
result_dir = os.path.join(os.path.expanduser('~'),"edskywalker","processed-data")

os.makedirs(download_dir, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

print(download_dir)

In [ ]:
list_flags_common = ['INLAND_WATER','CLOUD','CLOUD_AMBIGUOUS','CLOUD_MARGIN','INVALID','COSMETIC','SATURATED','SUSPECT','HISOLZEN','HIGHGLINT','SNOW_ICE']
list_flags_process = ['AC_FAIL','WHITECAPS','ADJAC','RWNEG_O2','RWNEG_O3','RWNEG_O4','RWNEG_O5','RWNEG_O6','RWNEG_O7','RWNEG_O8']
list_flags_oc4me = ['OC4ME_FAIL']
list_flags_ocnn = ['OCNN_FAIL']


def flag_data_fast(list_flag, flag_names, flag_values, flag_data, flag_type='WQSF'):
    flag_bits = np.uint64()
    if flag_type == 'SST':
        flag_bits = np.uint8()
    elif flag_type == 'WQSF_lsb':
        flag_bits = np.uint32()
    for flag in list_flag:
        try:
            flag_bits = flag_bits | flag_values[flag_names.index(flag)]
        except:
            print(flag + 'not present')
    return (flag_data & flag_bits) > 0			


In [ ]:
print("Please enter your [bold u cyan]username[/bold u cyan] and [bold u cyan]password[/bold u cyan]")

print()

while True:
    try:
        user = input("Enter your WEkEO username: ")
        passw = getpass("Enter your WEkEO password: ")
        c = hda.Client(hda.Configuration(user=user, password=passw), progress=True, max_workers=1)
        print()
        print(f"Login successfull!. Your token is {c.token}.")
        break
    except KeyError:
        print()
        print('You entered wrong username and/or password. Please repeat!')

In [ ]:
# Satellite parameter
print(
    Markdown("Please enter the :satellite: satellite parameters. For satellite ID, enter `1` to choose `EO:EUM:DAT:SENTINEL-3:0556` and `2` to choose `EO:EUM:DAT:SENTINEL-3:OL_2_WFR___`.")
)

print()

while True:
    sat_id = int(input('Satellite ID: '))

    if sat_id == 1:
        dataset_id = 'EO:EUM:DAT:SENTINEL-3:0556'
        print('Dataset ID: ', dataset_id)
        break
    elif sat_id == 2:
        dataset_id = 'EO:EUM:DAT:SENTINEL-3:OL_2_WFR___'
        print('Dataset ID', dataset_id)
        break
    else:
        print("You put wrong number. Please try again!")

In [ ]:
print(
    Markdown("For satellite name, enter `a` to choose `Sentinel-3A` and `b` to choose `Sentinel-3B`. Leave it blank if you want both Sentinel-3A and Sentinel-3B queried. Please note that Sentinel-3B id only available from 18 May 2018")
)

sat_nm = input("Satellite name: ")

if sat_nm == 'a':
    sat = 'Sentinel-3A'
    print('Satellite: ', sat)
elif sat_nm == 'b':
    sat = 'Sentinel-3B'
    print('Satellite: ', sat)
else:
    sat = ''
    print('Both Sentinel-3A and Sentinel-3B will be queried.')



In [ ]:
# Area of interest
print('Please input your :earth_asia-text: area of interest. The coordinates should be in [bold yellow]decimal format[/bold yellow] with :heavy_minus_sign-text: sign for south-of-equator latitude or west-of-greenwich longitude')

north = input('North point: ') # -6.85
south = input('South point: ') # -7.95
west = input('West point: ') # 112.66
east = input('East point: ') # 114.65

bbox = [west, south, east, north]
extent = [west, east, north, south]
bbox_str = f'{west},{east},{south},{north}' 

bbox_polygon = geometry.Polygon(((west,south), (west,north), (east,north), (east,south)))

In [ ]:
## Time of interest
print("Please input your :spiral_calendar-text: [u]date of interest[/u]. The dates should be in [bold yellow]YYYY-MM-DD[/bold yellow] format.")
print()
dtstart = input('Time start: ')
dtend = input('Time end: ')

In [ ]:
query = {
  "dataset_id": dataset_id, 
  "dtstart": dtstart,
  "dtend": dtend,
  "bbox": bbox,
  "sat": sat,
  "type": "OL_2_WFR___",
  "timeliness": "NT",
  "itemsPerPage": 200,
  "startIndex": 0
}


In [ ]:
search_result = c.search(query)
print(search_result)


In [ ]:
for index, res in tqdm(enumerate(search_result.results, start=0), desc='Processed', total=len(search_result.results)):
    file_id = res['id']
    
    search_result[index].download()

    with zipfile.ZipFile(file_id + '.zip', 'r') as zip_ref:
        zip_ref.extractall(download_dir)
        print(f'Unzipping of product {file_id} finished.')
        os.remove(file_id + '.zip')

    filedir = os.path.join(download_dir, file_id)
    geo_coords = xr.open_dataset(filedir + '/geo_coordinates.nc')
    lon = geo_coords.variables['longitude'].data
    lat = geo_coords.variables['latitude'].data

    flag_file = xr.open_dataset(filedir + '/wqsf.nc')
    flag_names = flag_file['WQSF'].flag_meanings.split(' ') #flag names
    flag_vals = flag_file['WQSF'].flag_masks #flag bit values
    flags_data = flag_file.variables['WQSF'].data
    
    chlnn = xr.open_dataset(filedir + '/chl_nn.nc')
    chloc = xr.open_dataset(filedir + '/chl_oc4me.nc')
    tsmnn = xr.open_dataset(filedir + '/tsm_nn.nc')
    
    geo_file = xr.combine_by_coords([chlnn, chloc, tsmnn], join="exact", combine_attrs="drop_conflicts")
    chl_nn = geo_file.variables['CHL_NN'].data

    ref_file = xr.open_mfdataset(filedir + '/*reflectance.nc')
    
    flag_mask = flag_data_fast(list_flag_nn, flag_names, flag_vals, flags_data, flag_type='WQSF') # return a numpy array with selected flags
    chl_flagged = np.where(flag_mask, np.nan, chl_nn) # return a numpy array of masked chl-a data

    start = datetime.strptime(res['properties']['startdate'], '%Y-%m-%dT%H:%M:%S%fZ')
    end = datetime.strptime(res['properties']['enddate'], '%Y-%m-%dT%H:%M:%S%fZ')
    timestamp = start + (end - start) / 2

    dta = xr.Dataset()
    
    dta['longitude'] = xr.DataArray(lon, dims=('rows','columns'))
    dta['longitude'].attrs = geo_coords['longitude'].attrs
    dta['latitude'] = xr.DataArray(lat, dims=('rows','columns'))
    dta['latitude'].attrs = geo_coords['latitude'].attrs
    
    dta['chl_flag'] = xr.DataArray(chl_flagged, dims=('rows','columns'))
    dta['chl_flag'].attrs = geo_file['CHL_NN'].attrs
    dta['chl_nn'] = xr.DataArray(chl_nn, dims=('rows','columns'))
    dta['chl_nn'].attrs = geo_file['CHL_NN'].attrs
    
    dta = dta.set_coords(['longitude','latitude'])
    
    dta = dta.expand_dims(dim={"time":[timestamp]}, axis=0)

    reggridded = cdo.sellonlatbox(bbox_str, input = dta, returnXDataset = True)
    reggridded.to_netcdf(os.path.join(result_dir , file_id + '.nc'))

    cdo.cleanTempDir()
    for allitem in os.listdir(download_dir):
        path = os.path.join(download_dir,allitem)
        if os.path.isfile(path):
            os.remove(path)
        elif os.path.isdir(path):
            shutil.rmtree(path)

    del geo_coords
    del geo_file
    del flag_file
    del dta

    gc.collect()